In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read files and cocatinating all of them
files = os.listdir('/kaggle/input/goodreads-book-datasets-10m')
all_data = []
for file in files:
    if file.startswith('book'):
        print(file)
        df= pd.read_csv('/kaggle/input/goodreads-book-datasets-10m/' + file ,usecols= ['Name','Authors','PublishYear','Rating','Language'] )
        df['ratings_k'] = file.split('.')[0]
        all_data.append(df)

In [ ]:
data = pd.concat(all_data,axis = 0)
data.info()

In [ ]:
data.head()

In [ ]:
data.ratings_k = data.ratings_k.apply(lambda x : x.split('book')[1])
data.head()

In [ ]:
data.ratings_k.value_counts(sort = False)

In [ ]:
data.ratings_k.astype('category',['1-100k','100-200k','200-300k','300-400k','400-500k',
                                  '500-600k','600-700k','700-800k','800-900k','900-1000k',
                                  '1000-1100k','1100-1200k','1200-1300k','1300-1400k'
                                 '1400-1500k','1500-1600k','1600-1700k','1700-1800k',
                                 '1800-1900k','1900-2000k','2000-3000k','3000-4000k',
                                 '4000-5000k'])

In [ ]:
data.loc[data.Name == "The Gift"]

In [ ]:
df = data.copy()

In [ ]:
df.PublishYear = df.PublishYear.astype(str)

In [ ]:
from datetime import datetime
df.PublishYear = df.PublishYear.apply(lambda s : datetime.strptime(s,"%Y").year if len(s)== 4 else np.NaN)
df.head()

In [ ]:
df.PublishYear  = df.PublishYear.apply(lambda x: x if x in range(1800,2021) else np.NaN)

In [ ]:
df.isna().sum()

In [ ]:
df.PublishYear.value_counts()

In [ ]:
data2 = pd.read_csv('/kaggle/input/goodreads-book-datasets-10m/user_rating_0_to_1000.csv', usecols=['Name','Rating'])
data2.head()

In [ ]:
data_merge = pd.merge(df, data2, on = 'Name', how = 'right')

In [ ]:
data_merge.PublishYear.fillna(method = 'bfill',inplace=True)

In [ ]:
data_merge.columns

In [ ]:
data_merge.dropna(inplace = True)

In [ ]:
data_merge.isna().any()

In [ ]:
data_merge.drop_duplicates(subset= ['Name'],inplace = True)
data_merge.head()

In [ ]:
data_merge.PublishYear.max()

In [ ]:
data_merge.PublishYear.min()

In [ ]:
new_data = data_merge[data_merge['PublishYear'] >= 1990]

In [ ]:
new_data.Rating_y.unique()

In [ ]:
dic = {'it was amazing':4.5,
      'really liked it':4.5,'liked it':3.8,
      'it was ok':3.5,
      'did not like it':2.0}
new_data['Rating_new'] = new_data['Rating_y'].map(dic)

In [ ]:
new_data['Rating_mean'] = ((new_data['Rating_x'] + new_data['Rating_new'])/2).round(2).astype(float)
new_data.head()

In [ ]:
new_data.drop(['Rating_x', 'Rating_y', 'Rating_new'], axis = 1, inplace = True)

In [ ]:
def top_rated(data):
    return data.sort_values(by = ['PublishYear','Rating_mean'],ascending = [False,False])[:1]

In [ ]:
best_book_overyears = new_data.groupby('PublishYear').apply(top_rated).reset_index(drop= True)

In [ ]:
best_book_overyears.head()